In [1]:
!pip install transformers
!pip install kaggle


In [ ]:
# Download the Kaggle dataset (replace with your dataset's name and path)
#!mkdir ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d 'mexwell/food-vitamins-minerals-macronutrient'

# Unzip the dataset (adjust the file name if needed)
#!unzip food-vitamins-minerals-macronutrient.zip
#!ls

In [ ]:
# Import necessary libraries
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import pickle

# Load the dataset into a pandas DataFrame
df = pd.read_csv('megaGymDataset.csv')

# Specify columns to remove
columns_to_remove = ['Rating', 'RatingDesc']

# Remove specified columns
df = df.drop(columns=columns_to_remove)

# Get text data from the second column (food ingredient names)
workouts = df.iloc[:, 1].tolist()

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=50)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()  # Get [CLS] token embeddings
    return embeddings

# Get BERT embeddings for exercises
ingredient_embeddings = [get_bert_embeddings(workout) for workout in workouts]

# Create a new DataFrame with exercises and their embeddings
embeddings_df = pd.DataFrame({'workout': workouts, 'embedding': ingredient_embeddings})



In [8]:
# Add embeddings to the preprocessed DataFrame
df['embedding'] = ingredient_embeddings

# Save the updated DataFrame to a pickle file
with open('data_with_embeddings.pkl', 'wb') as file:
    pickle.dump(df, file)

In [7]:
print("Preprocessed CSV:")
display(df.head())